In [548]:
# yr = 2001
# em_1hr

'01'

In [1]:
# # ### Get all files for this ensemble member
# general_filename_30mins = f'/nfs/a319/gy17m2a/PhD/datadir/UKCP18_first30mins/{yrs_range}/{em_30mins}/{em_30mins}a.pr{yr}{month_num}*'
# filenames_first30mins = []
# for filename_30mins in glob.glob(general_filename_30mins):
#     filenames_first30mins.append(filename_30mins)
    
# filenames_first30mins.sort()

# # ### Load in the data 
# monthly_cubes_list_30mins = iris.load(filenames_first30mins)
# monthly_cube_30mins = monthly_cubes_list_30mins.concatenate_cube()      

In [2]:
import iris
import os
import glob as glob
import datetime as datetime
import iris.coord_categorisation as cat
import sys
import iris.plot as iplt

# Set up path to root directory
root_fp = "/nfs/a319/gy17m2a/PhD/"
os.chdir(root_fp)

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)
from warnings import simplefilter
# ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)


# Create path to files containing functions
sys.path.insert(0, root_fp + 'Scripts/GlobalFunctions')
from Spatial_plotting_functions import *
from Spatial_geometry_functions import *


# ### Load necessary spatial data
# This is a square area surrounding Leeds
leeds_at_centre_gdf = create_leeds_at_centre_outline({'init' :'epsg:3857'})


# ### Establish the corresponding ensemble member numbers
em_matching_dict = {'01':'bc005', '04': 'bc006', '05': 'bc007', '06':'bc009',  '07':'bc010', 
                    '08': 'bc011', '09':'bc013', '10': 'bc015', '11': 'bc016', '12': 'bc017', '13':'bc018', '15':'bc012'}


yrs_range = "2001_2020"
# em_1hr = '05'
# yr = 2012
# month_num = '06'

for em_1hr in ['01','04', '05', '06',  '07','08', '09', '10', '11', '12', '13','15']:
    em_30mins = em_matching_dict[em_1hr]
    for yr in range(2001,2021):
        for month_num in ['06', '07', '08']:

            print(yr, em_1hr, em_30mins, month_num)

            ####################################################### 
            #######################################################
            ## Get one month of data - HOURLY
            ####################################################### 
            #######################################################
            ### Get a list of filenames for hourly data
            general_filename_1hr = f'/nfs/a319/gy17m2a/PhD/datadir/UKCP18_hourly/2.2km/{em_1hr}/{yrs_range}/pr_rcp85_land-cpm_uk_2.2km_{em_1hr}_1hr_{yr}{month_num}*'
            filenames_1hr = []
            for filename in glob.glob(general_filename_1hr):
                    filenames_1hr.append(filename)
            # If don't find any files matching this string in the 2001_2020 folder, then check the 1980_2001
            if len(filenames_1hr) == 0:
                general_filename_1hr = f'/nfs/a319/gy17m2a/PhD/datadir/UKCP18_hourly/2.2km/{em_1hr}/1980_2001/pr_rcp85_land-cpm_uk_2.2km_{em_1hr}_1hr_{yr}{month_num}*'
                for filename in glob.glob(general_filename_1hr):
                        filenames_1hr.append(filename)
                print(len(filenames_1hr))

            # ### Load in the data and remove the ensemble member dimension
            monthly_cubes_list_1hr = iris.load(filenames_1hr)
            cube_1hr = monthly_cubes_list_1hr[0]
            cube_1hr = cube_1hr[0,:,:,:]

            # ### Trim to Leeds
            cube_1hr = trim_to_bbox_of_region(cube_1hr, leeds_at_centre_gdf)

            #######################################################
            #######################################################
            ## Get one month of data - 30mins
            #######################################################
            #######################################################

            # ### Get all files for this ensemble member
            general_filename_30mins = f'/nfs/a319/gy17m2a/PhD/datadir/UKCP18_first30mins/{yrs_range}/{em_30mins}/{em_30mins}a.pr{yr}{month_num}*'
            filenames_first30mins = []
            for filename_30mins in glob.glob(general_filename_30mins):
                filenames_first30mins.append(filename_30mins)
            filenames_first30mins.sort()

            # ### Load in the data 
            monthly_cubes_list_30mins = iris.load(filenames_first30mins)
            monthly_cube_30mins = monthly_cubes_list_30mins.concatenate_cube()      

            # ### Trim to Leeds
            monthly_cube_30mins_1st = trim_to_bbox_of_region_30mins(monthly_cube_30mins, leeds_at_centre_gdf)


            # ### Convert units of 30 mins data
            # Check current units
            print(monthly_cube_30mins_1st.units)
            # Set the units to those of the 1 hr cube
            monthly_cube_30mins_1st.units = cube_1hr.units
            print(monthly_cube_30mins_1st.units)

            # Convert the data to also be this unit
            monthly_cube_30mins_1st_data = monthly_cube_30mins_1st.data
            monthly_cube_30mins_1st_data = monthly_cube_30mins_1st_data*3600

            monthly_cube_30mins_1st.data = monthly_cube_30mins_1st_data


            #######################################################
            #######################################################
            ## Find the second half of the hour, using the first half of hour and hourly values
            #######################################################
            #######################################################
            # get the hourly data
            cube_1hr_data = cube_1hr.data
            # calculate value for second half of hour
            second_half_of_the_hour_mean_hourly_rainfall_rate_data = 2*cube_1hr_data-monthly_cube_30mins_1st_data
            # Create a new cube for the second half of the hour (start by copying the first half of hour cube)
            monthly_cube_30mins_2nd = monthly_cube_30mins_1st.copy()
            # Set values as calculated
            monthly_cube_30mins_2nd.data = second_half_of_the_hour_mean_hourly_rainfall_rate_data


            # ### Edit the times to be 30 mins later
            # get the times from the first half hour
            first_half_hour_times = monthly_cube_30mins_1st.coord('time').copy()
            # add 30 mins
            second_half_hour_times = first_half_hour_times + 0.5
            # for the second hald hour cube, remove the time dimension and then re-add the edited one
            monthly_cube_30mins_2nd.remove_coord('time')
            monthly_cube_30mins_2nd.add_dim_coord(second_half_hour_times, 0)


            #######################################################
            #######################################################
            ## Join first half hour and second half hour into one cube
            #######################################################
            #######################################################

            # ### Get a list of all the cubes in each of the monthly cubes
            list_30mins_1st = iris.cube.CubeList(monthly_cube_30mins_1st.slices_over('time'))
            list_30mins_2nd = iris.cube.CubeList(monthly_cube_30mins_2nd.slices_over('time'))
            list_30mins = list_30mins_1st +  list_30mins_2nd


            ### Merge back into one cube
            monthly_cube_30mins =total_list.merge_cube()


            #######################################################
            #######################################################
            ## Save
            #######################################################
            #######################################################
            dir_to_save = f"datadir/UKCP18_every30mins/{em_30mins}/{yrs_range}/"

            if os.path.isdir(dir_to_save):
                print("Exists")
            else:
                print("Doesn't exist")
                os.makedirs(dir_to_save)
            fp_to_save = f"datadir/UKCP18_every30mins/{em_30mins}/{yrs_range}/{em_30mins}a.pr{yr}{month_num}.nc" 
            print(fp_to_save)
            iris.save(monthly_cube_30mins, fp_to_save)





2001 01 bc005 06
1
kg m-2 s-1
mm/hour


ValueError: operands could not be broadcast together with shapes (720,33,37) (624,33,37) 

In [8]:
filenames_first30mins

['/nfs/a319/gy17m2a/PhD/datadir/UKCP18_first30mins/2001_2020/bc005/bc005a.pr20010605.nc',
 '/nfs/a319/gy17m2a/PhD/datadir/UKCP18_first30mins/2001_2020/bc005/bc005a.pr20010606.nc',
 '/nfs/a319/gy17m2a/PhD/datadir/UKCP18_first30mins/2001_2020/bc005/bc005a.pr20010607.nc',
 '/nfs/a319/gy17m2a/PhD/datadir/UKCP18_first30mins/2001_2020/bc005/bc005a.pr20010608.nc',
 '/nfs/a319/gy17m2a/PhD/datadir/UKCP18_first30mins/2001_2020/bc005/bc005a.pr20010609.nc',
 '/nfs/a319/gy17m2a/PhD/datadir/UKCP18_first30mins/2001_2020/bc005/bc005a.pr20010610.nc',
 '/nfs/a319/gy17m2a/PhD/datadir/UKCP18_first30mins/2001_2020/bc005/bc005a.pr20010611.nc',
 '/nfs/a319/gy17m2a/PhD/datadir/UKCP18_first30mins/2001_2020/bc005/bc005a.pr20010612.nc',
 '/nfs/a319/gy17m2a/PhD/datadir/UKCP18_first30mins/2001_2020/bc005/bc005a.pr20010613.nc',
 '/nfs/a319/gy17m2a/PhD/datadir/UKCP18_first30mins/2001_2020/bc005/bc005a.pr20010614.nc',
 '/nfs/a319/gy17m2a/PhD/datadir/UKCP18_first30mins/2001_2020/bc005/bc005a.pr20010615.nc',
 '/nfs/a31

In [6]:
# cube_1hr_data.shape
cube_1hr

Lwe Precipitation Rate (mm/hour),time,grid_latitude,grid_longitude
Shape,720,33,37
Dimension coordinates,,,
time,x,-,-
grid_latitude,-,x,-
grid_longitude,-,-,x
Auxiliary coordinates,,,
month_number,x,-,-
year,x,-,-
yyyymmddhh,x,-,-
latitude,-,x,x


In [5]:
monthly_cube_30mins_1st_data.shape

(624, 33, 37)

### Set up environment

In [24]:
import iris
import os
import glob as glob
import datetime as datetime
import iris.coord_categorisation as cat
import sys
import iris.plot as iplt

# Set up path to root directory
root_fp = "/nfs/a319/gy17m2a/PhD/"
os.chdir(root_fp)

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)
from warnings import simplefilter
# ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)


# Create path to files containing functions
sys.path.insert(0, root_fp + 'Scripts/GlobalFunctions')
from Spatial_plotting_functions import *
from Spatial_geometry_functions import *

### Load necessary spatial data

In [6]:
# This is a square area surrounding Leeds
leeds_at_centre_gdf = create_leeds_at_centre_outline({'init' :'epsg:3857'})

### Establish the corresponding ensemble member numbers

In [7]:
em_matching_dict = {'01':'bc005', '04': 'bc006', '05': 'bc007', '06':'bc009',  '07':'bc010', 
                    '08': 'bc011', '09':'bc013', '10': 'bc015', '11': 'bc016', '12': 'bc017', '13':'bc018', '15':'bc012'}

In [482]:
em_1hr = '05'
yrs_range = "2001_2020"
em_30mins = em_matching_dict[em_1hr]
yr = 2012
month_num = '06'

####################################################### 
#######################################################
## Get one month of data - HOURLY
### Get a list of filenames for hourly data
####################################################### 
#######################################################

In [506]:
yrs_range = "2001_2020"
# em_1hr = '05'
# yr = 2012
# month_num = '06'

for em_1hr in ['01','04', '05', '06',  '07','08', '09', '10', '11', '12', '13','15']:
    em_30mins = em_matching_dict[em_1hr]
    for yr in range(1999,2021):
        for month_num in ['06', '07', '08']:

            print(yr, em_1hr, em_30mins, month_num)

            ####################################################### 
            #######################################################
            ## Get one month of data - HOURLY
            ### Get a list of filenames for hourly data
            ####################################################### 
            #######################################################
            general_filename_1hr = f'/nfs/a319/gy17m2a/PhD/datadir/UKCP18_hourly/2.2km/{em_1hr}/{yrs_range}/pr_rcp85_land-cpm_uk_2.2km_{em_1hr}_1hr_{yr}{month_num}*'
            print(general_filename_1hr)
            filenames_1hr = []
            for filename in glob.glob(general_filename_1hr):
                    filenames_1hr.append(filename)
            print(len(filenames_1hr))
            
            if len(filenames_1hr) == 0:
                general_filename_1hr = f'/nfs/a319/gy17m2a/PhD/datadir/UKCP18_hourly/2.2km/{em_1hr}/1980_2001/pr_rcp85_land-cpm_uk_2.2km_{em_1hr}_1hr_{yr}{month_num}*'
                print(general_filename_1hr)
                filenames_1hr = []
                for filename in glob.glob(general_filename_1hr):
                        filenames_1hr.append(filename)
                print(len(filenames_1hr))

1999 01 bc005 06
/nfs/a319/gy17m2a/PhD/datadir/UKCP18_hourly/2.2km/01/2001_2020/pr_rcp85_land-cpm_uk_2.2km_01_1hr_199906*
0
/nfs/a319/gy17m2a/PhD/datadir/UKCP18_hourly/2.2km/01/1980_2001/pr_rcp85_land-cpm_uk_2.2km_01_1hr_199906*
1
1999 01 bc005 07
/nfs/a319/gy17m2a/PhD/datadir/UKCP18_hourly/2.2km/01/2001_2020/pr_rcp85_land-cpm_uk_2.2km_01_1hr_199907*
0
/nfs/a319/gy17m2a/PhD/datadir/UKCP18_hourly/2.2km/01/1980_2001/pr_rcp85_land-cpm_uk_2.2km_01_1hr_199907*
1
1999 01 bc005 08
/nfs/a319/gy17m2a/PhD/datadir/UKCP18_hourly/2.2km/01/2001_2020/pr_rcp85_land-cpm_uk_2.2km_01_1hr_199908*
0
/nfs/a319/gy17m2a/PhD/datadir/UKCP18_hourly/2.2km/01/1980_2001/pr_rcp85_land-cpm_uk_2.2km_01_1hr_199908*
1
2000 01 bc005 06
/nfs/a319/gy17m2a/PhD/datadir/UKCP18_hourly/2.2km/01/2001_2020/pr_rcp85_land-cpm_uk_2.2km_01_1hr_200006*
0
/nfs/a319/gy17m2a/PhD/datadir/UKCP18_hourly/2.2km/01/1980_2001/pr_rcp85_land-cpm_uk_2.2km_01_1hr_200006*
1
2000 01 bc005 07
/nfs/a319/gy17m2a/PhD/datadir/UKCP18_hourly/2.2km/01/2001_2

1
1999 04 bc006 07
/nfs/a319/gy17m2a/PhD/datadir/UKCP18_hourly/2.2km/04/2001_2020/pr_rcp85_land-cpm_uk_2.2km_04_1hr_199907*
0
/nfs/a319/gy17m2a/PhD/datadir/UKCP18_hourly/2.2km/04/1980_2001/pr_rcp85_land-cpm_uk_2.2km_04_1hr_199907*
1
1999 04 bc006 08
/nfs/a319/gy17m2a/PhD/datadir/UKCP18_hourly/2.2km/04/2001_2020/pr_rcp85_land-cpm_uk_2.2km_04_1hr_199908*
0
/nfs/a319/gy17m2a/PhD/datadir/UKCP18_hourly/2.2km/04/1980_2001/pr_rcp85_land-cpm_uk_2.2km_04_1hr_199908*
1
2000 04 bc006 06
/nfs/a319/gy17m2a/PhD/datadir/UKCP18_hourly/2.2km/04/2001_2020/pr_rcp85_land-cpm_uk_2.2km_04_1hr_200006*
0
/nfs/a319/gy17m2a/PhD/datadir/UKCP18_hourly/2.2km/04/1980_2001/pr_rcp85_land-cpm_uk_2.2km_04_1hr_200006*
1
2000 04 bc006 07
/nfs/a319/gy17m2a/PhD/datadir/UKCP18_hourly/2.2km/04/2001_2020/pr_rcp85_land-cpm_uk_2.2km_04_1hr_200007*
0
/nfs/a319/gy17m2a/PhD/datadir/UKCP18_hourly/2.2km/04/1980_2001/pr_rcp85_land-cpm_uk_2.2km_04_1hr_200007*
1
2000 04 bc006 08
/nfs/a319/gy17m2a/PhD/datadir/UKCP18_hourly/2.2km/04/2001

1
1999 05 bc007 07
/nfs/a319/gy17m2a/PhD/datadir/UKCP18_hourly/2.2km/05/2001_2020/pr_rcp85_land-cpm_uk_2.2km_05_1hr_199907*
0
/nfs/a319/gy17m2a/PhD/datadir/UKCP18_hourly/2.2km/05/1980_2001/pr_rcp85_land-cpm_uk_2.2km_05_1hr_199907*
1
1999 05 bc007 08
/nfs/a319/gy17m2a/PhD/datadir/UKCP18_hourly/2.2km/05/2001_2020/pr_rcp85_land-cpm_uk_2.2km_05_1hr_199908*
0
/nfs/a319/gy17m2a/PhD/datadir/UKCP18_hourly/2.2km/05/1980_2001/pr_rcp85_land-cpm_uk_2.2km_05_1hr_199908*
1
2000 05 bc007 06
/nfs/a319/gy17m2a/PhD/datadir/UKCP18_hourly/2.2km/05/2001_2020/pr_rcp85_land-cpm_uk_2.2km_05_1hr_200006*
0
/nfs/a319/gy17m2a/PhD/datadir/UKCP18_hourly/2.2km/05/1980_2001/pr_rcp85_land-cpm_uk_2.2km_05_1hr_200006*
1
2000 05 bc007 07
/nfs/a319/gy17m2a/PhD/datadir/UKCP18_hourly/2.2km/05/2001_2020/pr_rcp85_land-cpm_uk_2.2km_05_1hr_200007*
0
/nfs/a319/gy17m2a/PhD/datadir/UKCP18_hourly/2.2km/05/1980_2001/pr_rcp85_land-cpm_uk_2.2km_05_1hr_200007*
1
2000 05 bc007 08
/nfs/a319/gy17m2a/PhD/datadir/UKCP18_hourly/2.2km/05/2001

1
1999 06 bc009 07
/nfs/a319/gy17m2a/PhD/datadir/UKCP18_hourly/2.2km/06/2001_2020/pr_rcp85_land-cpm_uk_2.2km_06_1hr_199907*
0
/nfs/a319/gy17m2a/PhD/datadir/UKCP18_hourly/2.2km/06/1980_2001/pr_rcp85_land-cpm_uk_2.2km_06_1hr_199907*
1
1999 06 bc009 08
/nfs/a319/gy17m2a/PhD/datadir/UKCP18_hourly/2.2km/06/2001_2020/pr_rcp85_land-cpm_uk_2.2km_06_1hr_199908*
0
/nfs/a319/gy17m2a/PhD/datadir/UKCP18_hourly/2.2km/06/1980_2001/pr_rcp85_land-cpm_uk_2.2km_06_1hr_199908*
1
2000 06 bc009 06
/nfs/a319/gy17m2a/PhD/datadir/UKCP18_hourly/2.2km/06/2001_2020/pr_rcp85_land-cpm_uk_2.2km_06_1hr_200006*
0
/nfs/a319/gy17m2a/PhD/datadir/UKCP18_hourly/2.2km/06/1980_2001/pr_rcp85_land-cpm_uk_2.2km_06_1hr_200006*
1
2000 06 bc009 07
/nfs/a319/gy17m2a/PhD/datadir/UKCP18_hourly/2.2km/06/2001_2020/pr_rcp85_land-cpm_uk_2.2km_06_1hr_200007*
0
/nfs/a319/gy17m2a/PhD/datadir/UKCP18_hourly/2.2km/06/1980_2001/pr_rcp85_land-cpm_uk_2.2km_06_1hr_200007*
1
2000 06 bc009 08
/nfs/a319/gy17m2a/PhD/datadir/UKCP18_hourly/2.2km/06/2001

1
1999 07 bc010 07
/nfs/a319/gy17m2a/PhD/datadir/UKCP18_hourly/2.2km/07/2001_2020/pr_rcp85_land-cpm_uk_2.2km_07_1hr_199907*
0
/nfs/a319/gy17m2a/PhD/datadir/UKCP18_hourly/2.2km/07/1980_2001/pr_rcp85_land-cpm_uk_2.2km_07_1hr_199907*
1
1999 07 bc010 08
/nfs/a319/gy17m2a/PhD/datadir/UKCP18_hourly/2.2km/07/2001_2020/pr_rcp85_land-cpm_uk_2.2km_07_1hr_199908*
0
/nfs/a319/gy17m2a/PhD/datadir/UKCP18_hourly/2.2km/07/1980_2001/pr_rcp85_land-cpm_uk_2.2km_07_1hr_199908*
1
2000 07 bc010 06
/nfs/a319/gy17m2a/PhD/datadir/UKCP18_hourly/2.2km/07/2001_2020/pr_rcp85_land-cpm_uk_2.2km_07_1hr_200006*
0
/nfs/a319/gy17m2a/PhD/datadir/UKCP18_hourly/2.2km/07/1980_2001/pr_rcp85_land-cpm_uk_2.2km_07_1hr_200006*
1
2000 07 bc010 07
/nfs/a319/gy17m2a/PhD/datadir/UKCP18_hourly/2.2km/07/2001_2020/pr_rcp85_land-cpm_uk_2.2km_07_1hr_200007*
0
/nfs/a319/gy17m2a/PhD/datadir/UKCP18_hourly/2.2km/07/1980_2001/pr_rcp85_land-cpm_uk_2.2km_07_1hr_200007*
1
2000 07 bc010 08
/nfs/a319/gy17m2a/PhD/datadir/UKCP18_hourly/2.2km/07/2001

...

1
2001 09 bc013 07
/nfs/a319/gy17m2a/PhD/datadir/UKCP18_hourly/2.2km/09/2001_2020/pr_rcp85_land-cpm_uk_2.2km_09_1hr_200107*
0
/nfs/a319/gy17m2a/PhD/datadir/UKCP18_hourly/2.2km/09/1980_2001/pr_rcp85_land-cpm_uk_2.2km_09_1hr_200107*
1
2001 09 bc013 08
/nfs/a319/gy17m2a/PhD/datadir/UKCP18_hourly/2.2km/09/2001_2020/pr_rcp85_land-cpm_uk_2.2km_09_1hr_200108*
0
/nfs/a319/gy17m2a/PhD/datadir/UKCP18_hourly/2.2km/09/1980_2001/pr_rcp85_land-cpm_uk_2.2km_09_1hr_200108*
1
2002 09 bc013 06
/nfs/a319/gy17m2a/PhD/datadir/UKCP18_hourly/2.2km/09/2001_2020/pr_rcp85_land-cpm_uk_2.2km_09_1hr_200206*
1
2002 09 bc013 07
/nfs/a319/gy17m2a/PhD/datadir/UKCP18_hourly/2.2km/09/2001_2020/pr_rcp85_land-cpm_uk_2.2km_09_1hr_200207*
1
2002 09 bc013 08
/nfs/a319/gy17m2a/PhD/datadir/UKCP18_hourly/2.2km/09/2001_2020/pr_rcp85_land-cpm_uk_2.2km_09_1hr_200208*
1
2003 09 bc013 06
/nfs/a319/gy17m2a/PhD/datadir/UKCP18_hourly/2.2km/09/2001_2020/pr_rcp85_land-cpm_uk_2.2km_09_1hr_200306*
1
2003 09 bc013 07
/nfs/a319/gy17m2a/PhD/d

1
2004 10 bc015 06
/nfs/a319/gy17m2a/PhD/datadir/UKCP18_hourly/2.2km/10/2001_2020/pr_rcp85_land-cpm_uk_2.2km_10_1hr_200406*
1
2004 10 bc015 07
/nfs/a319/gy17m2a/PhD/datadir/UKCP18_hourly/2.2km/10/2001_2020/pr_rcp85_land-cpm_uk_2.2km_10_1hr_200407*
1
2004 10 bc015 08
/nfs/a319/gy17m2a/PhD/datadir/UKCP18_hourly/2.2km/10/2001_2020/pr_rcp85_land-cpm_uk_2.2km_10_1hr_200408*
1
2005 10 bc015 06
/nfs/a319/gy17m2a/PhD/datadir/UKCP18_hourly/2.2km/10/2001_2020/pr_rcp85_land-cpm_uk_2.2km_10_1hr_200506*
1
2005 10 bc015 07
/nfs/a319/gy17m2a/PhD/datadir/UKCP18_hourly/2.2km/10/2001_2020/pr_rcp85_land-cpm_uk_2.2km_10_1hr_200507*
1
2005 10 bc015 08
/nfs/a319/gy17m2a/PhD/datadir/UKCP18_hourly/2.2km/10/2001_2020/pr_rcp85_land-cpm_uk_2.2km_10_1hr_200508*
1
2006 10 bc015 06
/nfs/a319/gy17m2a/PhD/datadir/UKCP18_hourly/2.2km/10/2001_2020/pr_rcp85_land-cpm_uk_2.2km_10_1hr_200606*
1
2006 10 bc015 07
/nfs/a319/gy17m2a/PhD/datadir/UKCP18_hourly/2.2km/10/2001_2020/pr_rcp85_land-cpm_uk_2.2km_10_1hr_200607*
1
2006 1

1
2007 11 bc016 08
/nfs/a319/gy17m2a/PhD/datadir/UKCP18_hourly/2.2km/11/2001_2020/pr_rcp85_land-cpm_uk_2.2km_11_1hr_200708*
1
2008 11 bc016 06
/nfs/a319/gy17m2a/PhD/datadir/UKCP18_hourly/2.2km/11/2001_2020/pr_rcp85_land-cpm_uk_2.2km_11_1hr_200806*
1
2008 11 bc016 07
/nfs/a319/gy17m2a/PhD/datadir/UKCP18_hourly/2.2km/11/2001_2020/pr_rcp85_land-cpm_uk_2.2km_11_1hr_200807*
1
2008 11 bc016 08
/nfs/a319/gy17m2a/PhD/datadir/UKCP18_hourly/2.2km/11/2001_2020/pr_rcp85_land-cpm_uk_2.2km_11_1hr_200808*
1
2009 11 bc016 06
/nfs/a319/gy17m2a/PhD/datadir/UKCP18_hourly/2.2km/11/2001_2020/pr_rcp85_land-cpm_uk_2.2km_11_1hr_200906*
1
2009 11 bc016 07
/nfs/a319/gy17m2a/PhD/datadir/UKCP18_hourly/2.2km/11/2001_2020/pr_rcp85_land-cpm_uk_2.2km_11_1hr_200907*
1
2009 11 bc016 08
/nfs/a319/gy17m2a/PhD/datadir/UKCP18_hourly/2.2km/11/2001_2020/pr_rcp85_land-cpm_uk_2.2km_11_1hr_200908*
1
2010 11 bc016 06
/nfs/a319/gy17m2a/PhD/datadir/UKCP18_hourly/2.2km/11/2001_2020/pr_rcp85_land-cpm_uk_2.2km_11_1hr_201006*
1
2010 1

1
2011 12 bc017 08
/nfs/a319/gy17m2a/PhD/datadir/UKCP18_hourly/2.2km/12/2001_2020/pr_rcp85_land-cpm_uk_2.2km_12_1hr_201108*
1
2012 12 bc017 06
/nfs/a319/gy17m2a/PhD/datadir/UKCP18_hourly/2.2km/12/2001_2020/pr_rcp85_land-cpm_uk_2.2km_12_1hr_201206*
1
2012 12 bc017 07
/nfs/a319/gy17m2a/PhD/datadir/UKCP18_hourly/2.2km/12/2001_2020/pr_rcp85_land-cpm_uk_2.2km_12_1hr_201207*
1
2012 12 bc017 08
/nfs/a319/gy17m2a/PhD/datadir/UKCP18_hourly/2.2km/12/2001_2020/pr_rcp85_land-cpm_uk_2.2km_12_1hr_201208*
1
2013 12 bc017 06
/nfs/a319/gy17m2a/PhD/datadir/UKCP18_hourly/2.2km/12/2001_2020/pr_rcp85_land-cpm_uk_2.2km_12_1hr_201306*
1
2013 12 bc017 07
/nfs/a319/gy17m2a/PhD/datadir/UKCP18_hourly/2.2km/12/2001_2020/pr_rcp85_land-cpm_uk_2.2km_12_1hr_201307*
1
2013 12 bc017 08
/nfs/a319/gy17m2a/PhD/datadir/UKCP18_hourly/2.2km/12/2001_2020/pr_rcp85_land-cpm_uk_2.2km_12_1hr_201308*
1
2014 12 bc017 06
/nfs/a319/gy17m2a/PhD/datadir/UKCP18_hourly/2.2km/12/2001_2020/pr_rcp85_land-cpm_uk_2.2km_12_1hr_201406*
1
2014 1

1
1999 15 bc012 07
/nfs/a319/gy17m2a/PhD/datadir/UKCP18_hourly/2.2km/15/2001_2020/pr_rcp85_land-cpm_uk_2.2km_15_1hr_199907*
0
/nfs/a319/gy17m2a/PhD/datadir/UKCP18_hourly/2.2km/15/1980_2001/pr_rcp85_land-cpm_uk_2.2km_15_1hr_199907*
1
1999 15 bc012 08
/nfs/a319/gy17m2a/PhD/datadir/UKCP18_hourly/2.2km/15/2001_2020/pr_rcp85_land-cpm_uk_2.2km_15_1hr_199908*
0
/nfs/a319/gy17m2a/PhD/datadir/UKCP18_hourly/2.2km/15/1980_2001/pr_rcp85_land-cpm_uk_2.2km_15_1hr_199908*
1
2000 15 bc012 06
/nfs/a319/gy17m2a/PhD/datadir/UKCP18_hourly/2.2km/15/2001_2020/pr_rcp85_land-cpm_uk_2.2km_15_1hr_200006*
0
/nfs/a319/gy17m2a/PhD/datadir/UKCP18_hourly/2.2km/15/1980_2001/pr_rcp85_land-cpm_uk_2.2km_15_1hr_200006*
1
2000 15 bc012 07
/nfs/a319/gy17m2a/PhD/datadir/UKCP18_hourly/2.2km/15/2001_2020/pr_rcp85_land-cpm_uk_2.2km_15_1hr_200007*
0
/nfs/a319/gy17m2a/PhD/datadir/UKCP18_hourly/2.2km/15/1980_2001/pr_rcp85_land-cpm_uk_2.2km_15_1hr_200007*
1
2000 15 bc012 08
/nfs/a319/gy17m2a/PhD/datadir/UKCP18_hourly/2.2km/15/2001

In [176]:
general_filename_1hr = f'datadir/UKCP18/2.2km/{em_1hr}/{yrs_range}/pr_rcp85_land-cpm_uk_2.2km_{em_1hr}_1hr_*'
filenames_1hr = []
filenames_june_1hr = []
for filename in glob.glob(general_filename_1hr):
    if filename[68:72] == str(yr): 
        if filename[72:74] in [month_num]:
            filenames_june_1hr.append(filename)
filenames_june_1hr

['datadir/UKCP18/2.2km/05/2001_2020/pr_rcp85_land-cpm_uk_2.2km_05_1hr_20120601-20120630.nc']

### Load in the data and remove the ensemble member dimension

In [403]:
monthly_cubes_list_1hr = iris.load(filenames_june_1hr)
cube_1hr = monthly_cubes_list_1hr[0]
cube_1hr = cube_1hr[0,:,:,:]

### Trim to Leeds

In [404]:
cube_1hr = trim_to_bbox_of_region(cube_1hr, leeds_at_centre_gdf)

#######################################################
#######################################################
## Get one month of data - 30mins
#######################################################
#######################################################

### Get all files for this ensemble member

In [405]:
general_filename_30mins = f'datadir/UKCP18_30mins/{em_30mins}/{em_30mins}a.pr*.pp'
filenames_30mins = []
for filename_30mins in glob.glob(general_filename_30mins):
    filenames_30mins.append(filename_30mins)
filenames_30mins.sort()

### Get just the filenames for June 2011

In [406]:
filenames_june = []
filenames_july =[]
filenames_august =[]
for filename in filenames_30mins:
    if filename[37:41] == str(yr):
            if filename[41:43] in [month_num]:
                filenames_june.append(filename)
#             if filename[41:43] in ['07']:   
#                 filenames_july.append(filename)  
#             if filename[41:43] in ['08']:    
#                 filenames_august.append(filename)

### Load in the data 

In [430]:
monthly_cubes_list_30mins = iris.load(filenames_june)
monthly_cube_30mins = monthly_cubes_list_30mins.concatenate_cube()      

### Trim to Leeds

In [408]:
monthly_cube_30mins_1st = trim_to_bbox_of_region_30mins(monthly_cube_30mins, leeds_at_centre_gdf)

### Convert units of 30 mins data

In [409]:
# Check current units
print(monthly_cube_30mins_1st.units)
# Set the units to those of the 1 hr cube
monthly_cube_30mins_1st.units = cube_1hr.units
print(monthly_cube_30mins_1st.units)

# Convert the data to also be this unit
monthly_cube_30mins_1st_data = monthly_cube_30mins_1st.data
monthly_cube_30mins_1st_data = monthly_cube_30mins_1st_data*3600

monthly_cube_30mins_1st.data = monthly_cube_30mins_1st_data

kg m-2 s-1
mm/hour


#######################################################
#######################################################
## Find the second half of the hour, using the first half of hour and hourly values
#######################################################
#######################################################

In [410]:
# get the hourly data
cube_1hr_data = cube_1hr.data
# calculate value for second half of hour
second_half_of_the_hour_mean_hourly_rainfall_rate_data = 2*cube_1hr_data-monthly_cube_30mins_1st_data
# Create a new cube for the second half of the hour (start by copying the first half of hour cube)
monthly_cube_30mins_2nd = monthly_cube_30mins_1st.copy()
# Set values as calculated
monthly_cube_30mins_2nd.data = second_half_of_the_hour_mean_hourly_rainfall_rate_data

### Edit the times to be 30 mins later

In [411]:
# get the times from the first half hour
first_half_hour_times = monthly_cube_30mins_1st.coord('time').copy()
# add 30 mins
second_half_hour_times = first_half_hour_times + 0.5
# for the second hald hour cube, remove the time dimension and then re-add the edited one
monthly_cube_30mins_2nd.remove_coord('time')
monthly_cube_30mins_2nd.add_dim_coord(second_half_hour_times, 0)

#######################################################
#######################################################
## Join first half hour and second half hour into one cube
#######################################################
#######################################################

### Get a list of all the cubes in each of the monthly cubes

In [478]:
list_30mins_1st = iris.cube.CubeList(monthly_cube_30mins_1st.slices_over('time'))
list_30mins_2nd = iris.cube.CubeList(monthly_cube_30mins_2nd.slices_over('time'))
list_30mins = list_30mins_1st +  list_30mins_2nd

### Merge back into one cube

In [476]:
monthly_cube_30mins =total_list.merge_cube()

#######################################################
#######################################################
## Save
#######################################################
#######################################################

In [491]:
dir_to_save = f"datadir/UKCP18_every30mins/{em_30mins}/{yrs_range}/"

if os.path.isdir(dir_to_save):
    print("Exists")
else:
    print("Doesn't exist")
    os.makedirs(dir_to_save)
fp_to_save = f"datadir/UKCP18_every30mins/{em_30mins}/{yrs_range}/{em_30mins}a.pr{yr}{month_num}.nc" 

Exists


In [492]:
iris.save(monthly_cube_30mins, fp_to_save)

## Finding the difference cube doesn't work because of
"ValueError: Coordinate 'time' has different points for the LHS cube 'lwe_precipitation_rate' and RHS cube 'stratiform_rainfall_flux'."  
Because 30 mins is .25 past hour and 1hr is .5 past the hour  
So set the times on 30 mins to be the same as for 1hr

### Finding the difference cube doesn't work because of
"ValueError: Coordinate 'grid_latitude' has different bounds for the LHS cube 'lwe_precipitation_rate' and RHS cube 'stratiform_rainfall_flux'"  
Because 30 mins is .25 past hour and 1hr is .5 past the hour  
So set the times on 30 mins to be the same as for 1hr

In [493]:
# monthly_cube_30mins.coord('grid_latitude').bounds =  cube_1hr.coord('grid_latitude').bounds
# monthly_cube_30mins.coord('grid_longitude').bounds =  cube_1hr.coord('grid_longitude').bounds